In [2]:
import pandas as pd
import numpy as np
import sklearn
%matplotlib inline
import matplotlib.pyplot as plt

Import data training data set

In [3]:
#os.chdir('D:\Sunny_Personal\OSU MSBA\SpecialCase')
blockbusters_Train=pd.read_excel('/content/Training sheet.xlsx')
blockbusters_Train.head()

,id,name,display_name,production_year,movie_sequel,creative_type,source,production_method,genre,language,board_rating_reason,movie_board_rating_display_name,movie_release_pattern_display_name,total,Category
0,7950115,Avatar,Avatar,2009,0,Science Fiction,Original Screenplay,Animation/Live Action,Action,English,"For intense epic battle sequences and warfare,...",PG-13,Wide,2784,9
1,50950115,Harry Potter and the Deathly Hallows: Part II,Harry Potter and the Deathly Hallows: Part II,2011,1,Fantasy,Based on Fiction Book/Short Story,Animation/Live Action,Adventure,English,for some sequences of intense action violence ...,PG-13,Wide,1328,9
2,119870115,Transformers 3,Transformers: Dark of the Moon,2011,1,Science Fiction,Based on TV,Animation/Live Action,Action,English,for intense prolonged sequences of sci-fi acti...,PG-13,Wide,1124,9
3,119590115,Toy Story 3,Toy Story 3,2010,1,Kids Fiction,Original Screenplay,Digital Animation,Adventure,English,General,G,Wide,1064,9
4,91700115,Pirates of the Caribbean 4,Pirates of the Caribbean: On Stranger Tides,2011,1,Fantasy,Based on Theme Park Ride,Live Action,Adventure,English,for intense sequences of action/adventure viol...,PG-13,Wide,1044,9


Understand the data - Shape, data types, EDA

In [4]:
#Data set no of rows and columns
blockbusters_Train.shape

(1196, 15)

In [5]:
#Data types and non null counts
blockbusters_Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  1196 non-null   int64 
 1   name                                1196 non-null   object
 2   display_name                        1196 non-null   object
 3   production_year                     1196 non-null   int64 
 4   movie_sequel                        1196 non-null   int64 
 5   creative_type                       1196 non-null   object
 6   source                              1196 non-null   object
 7   production_method                   1196 non-null   object
 8   genre                               1196 non-null   object
 9   language                            1196 non-null   object
 10  board_rating_reason                 1196 non-null   object
 11  movie_board_rating_display_name     1196 non-null   obje

In [6]:
#Any nulls in each column of Data frame ? (Cross examination)
blockbusters_Train.isnull().sum()

id                                    0
name                                  0
display_name                          0
production_year                       0
movie_sequel                          0
creative_type                         0
source                                0
production_method                     0
genre                                 0
language                              0
board_rating_reason                   0
movie_board_rating_display_name       0
movie_release_pattern_display_name    0
total                                 0
Category                              0
dtype: int64

In [7]:
#study the Continuous variables. Except for total earnings column, others dont mean anything
blockbusters_Train.describe(include='all')

,id,name,display_name,production_year,movie_sequel,creative_type,source,production_method,genre,language,board_rating_reason,movie_board_rating_display_name,movie_release_pattern_display_name,total,Category
count,1.196000e+03,1196,1196,1196.000000,1196.000000,1196,1196,1196,1196,1196,1196,1196,1196,1196.000000,1196.000000
unique,NaN,1196,1195,NaN,NaN,9,16,6,13,16,1003,6,7,NaN,NaN
top,NaN,"Uninvited, The",Death at a Funeral,NaN,NaN,Contemporary Fiction,Original Screenplay,Live Action,Drama,English,International - to be excluded,R,Wide,NaN,NaN
freq,NaN,1,2,NaN,NaN,638,629,1093,321,1144,83,448,795,NaN,NaN
mean,8.928203e+07,NaN,NaN,2008.984950,0.097826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.703177,3.564381
std,4.832893e+07,NaN,NaN,1.383625,0.297204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181.927715,1.962417
min,7.011500e+04,NaN,NaN,2007.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
25%,4.808012e+07,NaN,NaN,2008.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.000000,2.000000
50%,9.391012e+07,NaN,NaN,2009.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.500000,3.000000
75%,1.354326e+08,NaN,NaN,2010.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.250000,5.000000


In [8]:
#skewness of continuous variables
blockbusters_Train.skew(axis=0)  

id                -0.211269
production_year   -0.026172
movie_sequel       2.710920
total              4.990474
Category           0.730042
dtype: float64

In [9]:
#Checking the no of rows for each class in categorical variable columns
categorical = blockbusters_Train.iloc[:,[5,6,7,8,9,11,12,14]]
for c in categorical.columns:
    print("\n")
    print("\n")
    print(c)
    print(categorical[c].value_counts())





creative_type
Contemporary Fiction       638
Fantasy                    131
Historical Fiction         100
Dramatization               95
Science Fiction             89
Factual                     60
Kids Fiction                59
Super Hero                  20
Multiple Creative Types      4
Name: creative_type, dtype: int64




source
Original Screenplay                    629
Based on Fiction Book/Short Story      218
Based on Real Life Events              128
Remake                                  65
Based on TV                             38
Based on Comic/Graphic Novel            36
Based on Factual Book/Article           23
Based on Play                           21
Based on Folk Tale/Legend/Fairytale     10
Based on Game                            8
Spin-Off                                 5
Based on Short Film                      4
Based on Musical or Opera                4
Compilation                              3
Based on Theme Park Ride                 2
Based on Toy 

#Few inferences from above :
1) Language column is majorly english. this wont have much predictive power. 

In [10]:
#Correlation btw Gross earnings and Target CAtegory column. A value close to 1 means high correlation
corr_target = blockbusters_Train['total'].corr(blockbusters_Train['Category'])
corr_target

0.8054966083826659

#Gross earnings is the direct affect of the independent variables, and the category is manually created for analysis. But because of high correlation, we can safely consider the Category variable as target here for modeling

Before model building, let's try to tap the text data to extract information. Most occured words when segregated, may prove valuable

In [11]:
# filter stop words based on regular english stop words along with unique words which dont repeat much
#Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
#The board rating reason has several inconsistencies in the text. Manually correcting the spellings etc. This can also be done using TextBlob package
blockbusters_Train.loc[blockbusters_Train['movie_board_rating_display_name'] == 'Not Rated', 'board_rating_reason'] = 'foreign'
board_reason = blockbusters_Train['board_rating_reason']
board_reason = board_reason.str.replace('child','children')
board_reason = board_reason.str.replace('teen drinking','Underage_drinking')
board_reason = board_reason.str.replace('teen','teens')
board_reason = board_reason.str.replace('preteens','teens')
board_reason = board_reason.str.replace('nudtity','nudity')
board_reason = board_reason.str.replace('violents','violent')
board_reason = board_reason.str.replace('sexual attack','molest')
board_reason.head()

0    For intense epic battle sequences and warfare,...
1    for some sequences of intense action violence ...
2    for intense prolonged sequences of sci-fi acti...
3                                              General
4    for intense sequences of action/adventure viol...
Name: board_rating_reason, dtype: object

In [13]:
board_reason = board_reason.str.replace('sci-fi action violence','scifi_action_violence')
board_reason = board_reason.str.replace('sci-fi action and violence','scifi_action_violence')
board_reason = board_reason.str.replace('sci-fi action','scifi_action')
board_reason = board_reason.str.replace('sci-fi violence','scifi_violence')
board_reason = board_reason.str.replace('non-stop','non-stop')
board_reason.head()

0    For intense epic battle sequences and warfare,...
1    for some sequences of intense action violence ...
2    for intense prolonged sequences of scifi_actio...
3                                              General
4    for intense sequences of action/adventure viol...
Name: board_rating_reason, dtype: object

In [14]:
board_reason = board_reason.str.replace('sexaul','sexual')
board_reason = board_reason.str.replace('PG-13','')
board_reason = board_reason.str.replace('PG','')
board_reason = board_reason.str.replace('13','')
board_reason = board_reason.str.replace('arts','')
board_reason = board_reason.str.replace('martial','martial_arts')
board_reason = board_reason.str.replace('-','_')
board_reason = board_reason.str.replace('kids','children')
board_reason

0       For intense epic battle sequences and warfare,...
1       for some sequences of intense action violence ...
2       for intense prolonged sequences of scifi_actio...
3                                                 General
4       for intense sequences of action/adventure viol...
                              ...                        
1191    for strong violence/disturbing content, some s...
1192       for language, some violence and sexual content
1193                                              foreign
1194                                              foreign
1195    for mature thematic content including some sex...
Name: board_rating_reason, Length: 1196, dtype: object

In [15]:
English_Stop_Words = text.ENGLISH_STOP_WORDS
Board_Rating_Stop_Words = English_Stop_Words.union(["mild", "crude", "content", "partial", "elements", "comments", "strong", "use", "graphic", 
                                               "mature", "pervasive","and","for","involving", "situations", "caterpillar", "brief",  
                                               "rated", "frequent", "sustained", "pending", "momentary", "re_rated", "image", "edits",
                                               "scene", "situation", "audiences", "inter", "moments", "edited", "version", "originally", 
                                               "images", "including", "references", "suggestive", "thematic", "material","general","intense",
                                               "coarse", "extended", "edition", "realistic", "dequences", "moment", "materials", "descriptions"
                                               ])
vectorizer =  CountVectorizer(stop_words=Board_Rating_Stop_Words)
X = vectorizer.fit_transform(board_reason)
print(X)

  (0, 50)	1
  (0, 14)	1
  (0, 123)	1
  (0, 152)	1
  (0, 121)	1
  (0, 81)	1
  (0, 132)	1
  (1, 123)	1
  (1, 3)	1
  (1, 149)	1
  (1, 61)	1
  (2, 123)	1
  (2, 81)	1
  (2, 100)	1
  (2, 118)	1
  (2, 86)	1
  (2, 39)	1
  (2, 127)	1
  (2, 76)	1
  (4, 123)	1
  (4, 121)	1
  (4, 3)	1
  (4, 149)	1
  (4, 61)	1
  (4, 76)	1
  :	:
  (1184, 96)	1
  (1185, 81)	1
  (1185, 126)	1
  (1185, 42)	1
  (1185, 96)	1
  (1185, 150)	1
  (1185, 20)	1
  (1185, 11)	1
  (1186, 58)	1
  (1187, 81)	1
  (1187, 126)	1
  (1188, 81)	1
  (1188, 126)	1
  (1189, 149)	1
  (1191, 81)	1
  (1191, 149)	1
  (1191, 126)	1
  (1191, 42)	1
  (1191, 96)	1
  (1192, 81)	1
  (1192, 149)	1
  (1192, 126)	1
  (1193, 58)	1
  (1194, 58)	1
  (1195, 126)	1


In [16]:
print(X.shape)

(1196, 155)


In [17]:
# print the vocabulary
print(vectorizer.vocabulary_)

{'epic': 50, 'battle': 14, 'sequences': 123, 'warfare': 152, 'sensuality': 121, 'language': 81, 'smoking': 132, 'action': 3, 'violence': 149, 'frightening': 61, 'prolonged': 100, 'scifi_action_violence': 118, 'mayhem': 86, 'destruction': 39, 'sexuality': 127, 'innuendo': 76, 'adventure': 5, 'fantasy': 55, 'scary': 114, 'menace': 87, 'rude': 112, 'humor': 72, 'peril': 98, 'sexual': 126, 'drug': 45, 'swashbuckling': 140, 'disaster': 41, 'disturbing': 42, 'nudity': 96, 'martial_arts': 84, 'violent': 150, 'teenss': 142, 'sex_related': 125, 'adventures': 6, 'irreverent': 78, 'startling': 134, 'color': 29, 'scifi_violence': 119, 'emotional': 49, 'scifi_action': 117, 'drunkenness': 48, 'bullying': 22, 'bloody': 17, 'war': 151, 'male': 83, 'rear': 106, 'sci_fi': 116, 'western': 153, 'choreography': 27, 'foreign': 58, 'horror': 71, 'comic': 31, 'brutal': 20, 'rape': 104, 'torture': 145, 'alcohol': 7, 'reference': 108, 'gangster': 62, 'appeal': 10, 'teens': 141, 'underage_drinking': 147, 'accide

In [18]:
#Dataframe with features extracted from text data
text_features = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
blockbusters_Train.pop('board_rating_reason')
#Concatenating orginal data and the word counts dataframe
merged_Training_Data = pd.concat([blockbusters_Train,text_features],axis=1)
#Dropping a few columns - Total because we are considering Category as our target, name and displayname are just names, production_year is a 
#categorical variable which keeps changing every year (It can be explored using Panel regression to capture year wise variance.)
merged_Training_Data.drop(['total','id','name','display_name','production_year'], axis=1, inplace=True)
merged_Training_Data.head()

,movie_sequel,creative_type,source,production_method,genre,language,movie_board_rating_display_name,movie_release_pattern_display_name,Category,abuse,accident,accidents,action,admission,adventure,adventures,alcohol,animal,animated,appeal,assault,assaults,atrocities,battle,bawdy,behavior,bloody,bodily,bouts,brutal,brutality,bullying,carnage,cartoon,children,childrenren,choreography,climbing,color,combat,...,scenes,sci_fi,scifi_action,scifi_action_violence,scifi_violence,sensual,sensuality,sequence,sequences,sex,sex_related,sexual,sexuality,shooting,slang,slapstick,slavery,smoking,sports,startling,stunts,stylized,substance,suicide,supernatural,swashbuckling,teens,teenss,terror,thoughout,torture,underage,underage_drinking,vicious,violence,violent,war,warfare,western,zombie
0,0,Science Fiction,Original Screenplay,Animation/Live Action,Action,English,PG-13,Wide,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,Fantasy,Based on Fiction Book/Short Story,Animation/Live Action,Adventure,English,PG-13,Wide,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,1,Science Fiction,Based on TV,Animation/Live Action,Action,English,PG-13,Wide,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,Kids Fiction,Original Screenplay,Digital Animation,Adventure,English,G,Wide,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,Fantasy,Based on Theme Park Ride,Live Action,Adventure,English,PG-13,Wide,9,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [19]:
merged_Training_Data = merged_Training_Data.replace(' ', '_', regex=True)

Before Model building, categorical variables need to be one-hot encoded as they cannot be directly used in model. This can increase dimensions drastically
when we have lot of columns. So need to use judiciously. Here we have only 6 columns.

In [20]:
creative_type = pd.get_dummies(merged_Training_Data['creative_type'])
source = pd.get_dummies(merged_Training_Data['source'])
production_method = pd.get_dummies(merged_Training_Data['production_method'])
genre = pd.get_dummies(merged_Training_Data['genre'])
language = pd.get_dummies(merged_Training_Data['language'])
movie_board_rating_display_name = pd.get_dummies(merged_Training_Data['movie_board_rating_display_name'])
movie_release_pattern_display_name = pd.get_dummies(merged_Training_Data['movie_release_pattern_display_name'])

In [21]:
#show the one hot encoding
creative_type  

,Contemporary_Fiction,Dramatization,Factual,Fantasy,Historical_Fiction,Kids_Fiction,Multiple_Creative_Types,Science_Fiction,Super_Hero
0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1191,1,0,0,0,0,0,0,0,0
1192,1,0,0,0,0,0,0,0,0
1193,0,0,0,0,0,0,1,0,0
1194,1,0,0,0,0,0,0,0,0


In [22]:
#Joined dummy coding with original data. Then drop original variables
merged_dummies_training = pd.concat([merged_Training_Data,creative_type,source,production_method,genre,language,movie_board_rating_display_name,movie_release_pattern_display_name],axis=1)
merged_dummies_training.drop(['creative_type','source','production_method','genre','language','movie_board_rating_display_name','movie_release_pattern_display_name'], axis=1, inplace=True)

In [23]:
merged_dummies_training.head()

,movie_sequel,Category,abuse,accident,accidents,action,admission,adventure,adventures,alcohol,animal,animated,appeal,assault,assaults,atrocities,battle,bawdy,behavior,bloody,bodily,bouts,brutal,brutality,bullying,carnage,cartoon,children,childrenren,choreography,climbing,color,combat,comic,creature,criminality,cultural,dance,dancing,dangerous,...,Black_Comedy,Comedy,Concert/Performance,Documentary,Drama,Horror,Multiple_Genres,Musical,Romantic_Comedy,Thriller/Suspense,Western,language_Arabic,language_Danish,language_English,language_Farsi,language_French,language_German,language_Hebrew,language_Hindi,language_Italian,language_Japanese,language_Norwegian,language_Polish,language_Portuguese,language_Silent,language_Spanish,language_Swedish,G,NC-17,Not_Rated,PG,PG-13,R,Exclusive,Expands_Wide,IMAX,Limited,Oscar_Qualifying_Run,Special_Engagement,Wide
0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,1,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,1,9,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


The target variable has differnt no of observations in each class. This is called an unbalanced data set. Building predictive models on such data sets will introduce bias in the results. Few ways to overcome this problem is collect more data, resampling, Undersampling, Over sampling and if nothing else works we can change evaluation metric for assessing results. Here, we used a technique called SMOTE

In [24]:
#SMOTE - Synthetic Minority Over-sampling Technique
y_category = merged_dummies_training['Category']
merged_dummies_training.drop(['Category'], axis=1, inplace=True)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12)
x_res, y_res = sm.fit_sample(merged_dummies_training, y_category)
pd.Series(y_res).value_counts()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

9    247
7    247
5    247
3    247
1    247
8    247
6    247
4    247
2    247
dtype: int64

Random Forest to select important features. RF is a decision trees put together using Bagging, with added importance of feature selection

In [25]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
select = SelectFromModel(RandomForestClassifier(n_estimators=1000, random_state=42, min_samples_split = 15), threshold='mean')
select.fit(x_res, y_res)
x_res
X_featureSelct = select.transform(x_res)
X_featureSelct
print('The shape of X_train is: ', x_res.shape)
print('The shape of X_train_s is ', X_featureSelct.shape)

The shape of X_train is:  (2223, 228)
The shape of X_train_s is  (2223, 52)


Split the data set into training and Validation data sets. We train the models using training while measure the fit on validation data(test data here) to check if the model is overfitting.

In [26]:
#Splitting data in train and test
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_featureSelct, y_res, stratify=y_res, random_state=42, test_size = 0.25)

 1. Grid Search can be thought of as an exhaustive search for selecting a model. In Grid Search, the data scientist sets up a grid of hyperparameter values and for each combination, trains a model and scores on the testing data. In this approach, every combination of hyperparameter values is tried which can be very inefficient.

2. Random Search sets up a grid of hyperparameter values and selects random combinations to train the model and score.




In [ ]:
#Grid search on Logistic Regression - multinomial

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

grid={"C":np.logspace(-3,3,7), "penalty":["l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial')
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train, y_train)
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

In [89]:
from sklearn.linear_model import LogisticRegression
logreg2=LogisticRegression(C=100.0,penalty="l2",solver = 'lbfgs', multi_class = 'multinomial',max_iter = 1000)
logreg2.fit(X_train,y_train)
print("Train accuracy",logreg2.score(X_train,y_train))
print("Test accuracy",logreg2.score(X_test,y_test))

Train accuracy 0.528494301139772
Test accuracy 0.4118705035971223


In [ ]:
#Creating the random grid
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,10]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [5, 15, 26, 36, 47, 57, 68, 78, 89, 99, 110, None], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 4, 10], 'bootstrap': [True, False]}


Fit a random forest regressor using random grid and fetch the best hyperparameters

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 11.6min finished


{'bootstrap': False,
 'max_depth': 99,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 800}

In [ ]:
#using the parameters from Random search as a starting point narrowing down the best parameters using gridsearch

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': ['True','False'],
    'max_depth': [70,100,120],
    'min_samples_leaf': [1,2,5],
    'min_samples_split': [3,5,10],
    'n_estimators': [700,800,900]
}
# Create a based model
rf = RandomForestRegressor(max_features = 'sqrt')
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 162 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  9.5min finished


{'bootstrap': 'True',
 'max_depth': 70,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 900}

In [ ]:
best_grid = grid_search.best_estimator_
print(best_grid)

RandomForestRegressor(bootstrap='True', ccp_alpha=0.0, criterion='mse',
                      max_depth=70, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=900, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)


In [27]:
# Using the best hyper parameters the RF regressor has resulted in

forest = RandomForestClassifier(n_estimators=900, random_state=42, min_samples_split = 7,bootstrap = True, max_depth = 70,max_features='sqrt',min_samples_leaf = 1)
forest.fit(X_train, y_train)

print('Accuracy on the training subset: {:.3f}'.format(forest.score(X_train, y_train)))
print('Accuracy on the test subset: {:.3f}'.format(forest.score(X_test, y_test)))

Accuracy on the training subset: 0.860
Accuracy on the test subset: 0.563


## The accuracy above on the test data set has beaten the best accuracy rate from academic research paper given in the email

In [ ]:
#Applying grid search to identify best parameters for polynomial kernal based SVM"
from sklearn import svm 
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y, nfolds):
    Cs = [0.1, 10, 100, 200,1000]
    gammas = [0.05, 0.1,2,5,10]
    degrees = [3,4,5,6,7]
    param_grid = {'C': Cs, 'gamma' : gammas, 'degree' : degrees}
    grid_search = GridSearchCV(svm.SVC(kernel='poly'), param_grid, cv=nfolds)
    grid_search.fit(X_train, y_train)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
svc_param_selection(X_train, y_train, 3)

In [ ]:
grid_search.best_params_

In [94]:
# training a linear SVM classifier 
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'poly', degree = 5, gamma = 1, decision_function_shape = 'ovo',random_state = 42, C = 5000).fit(X_train, y_train) 
#svm_predictions = svm_model_linear.predict(X_test) 

accuracy_train_svm = svm_model_linear.score(X_train, y_train) 
print(accuracy_train_svm)

# accuracy on X_test 
accuracy_test_svm = svm_model_linear.score(X_test, y_test) 
print(accuracy_test_svm)

  
# creating a confusion matrix 
#cm_svm = confusion_matrix(y_test, svm_predictions)

0.9082183563287343
0.5287769784172662


In [93]:
# Gamma = Scale option works beter than Grid search
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'poly', degree = 6, gamma = 'scale', decision_function_shape = 'ovo',random_state = 42, C = 5000).fit(X_train, y_train) 
#svm_predictions = svm_model_linear.predict(X_test) 

accuracy_train_svm = svm_model_linear.score(X_train, y_train) 
print(accuracy_train_svm)

# accuracy on X_test 
accuracy_test_svm = svm_model_linear.score(X_test, y_test) 
print(accuracy_test_svm)


0.9082183563287343
0.5359712230215827


In [ ]:
#Trying grid search on Neural networks for identifying best parameters
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=50)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50),(150,150,150), (50),(150)],
    'activation': ['tanh', 'logistic','relu'],
    'solver': ['lbfgs','sgd', 'adam'],
    'alpha': [0.0001, 0.05, 0.1],
    'learning_rate': ['constant','adaptive'],
}
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=4)
clf.fit(X_train,y_train)

In [ ]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

In [92]:
#Fitting the NN model to the best parameters

#Best parameters found:
#{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (150, 150, 150), 'learning_rate': 'constant', 'solver': 'adam'}

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(activation = 'tanh',alpha = 0.05 ,solver = 'adam', hidden_layer_sizes=(150, 150, 150),learning_rate = 'constant', max_iter=500)

mlp.fit(X_train,y_train)

accuracy_train_mlp = mlp.score(X_train, y_train) 
print(accuracy_train_mlp)

# accuracy on X_test 
accuracy_test_mlp = mlp.score(X_test, y_test) 
print(accuracy_test_mlp)

0.9034193161367726
0.5251798561151079


In [28]:
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
gbm_ensemble = GradientBoostingClassifier(n_estimators=800, random_state=42, min_samples_split = 5,max_depth = 100,max_features='sqrt',min_samples_leaf = 1)
gbm_ensemble.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=100,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=5,
                           min_weight_fraction_leaf=0.0, n_estimators=800,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [90]:
gbm_ensemble.score(X_train,y_train)

0.9082183563287343

In [29]:

gbm_ensemble.score(X_test,y_test)

0.5467625899280576

In [30]:
scoring_Sheet = pd.read_excel('/content/Scoring Sheet.xlsx')

In [31]:
#Data set no of rows and columns
scoring_Sheet.shape

(91, 14)

In [32]:
#Data types and non null counts
scoring_Sheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  91 non-null     int64 
 1   name                                91 non-null     object
 2   display_name                        91 non-null     object
 3   production_budget                   91 non-null     int64 
 4   production_year                     91 non-null     int64 
 5   movie_sequel                        91 non-null     int64 
 6   creative_type                       91 non-null     object
 7   source                              91 non-null     object
 8   production_method                   91 non-null     object
 9   genre                               91 non-null     object
 10  language                            91 non-null     object
 11  board_rating_reason                 91 non-null     object
 

In [33]:
#Any nulls in each column of Data frame ? (Cross examination)
scoring_Sheet.isnull().sum()

id                                    0
name                                  0
display_name                          0
production_budget                     0
production_year                       0
movie_sequel                          0
creative_type                         0
source                                0
production_method                     0
genre                                 0
language                              0
board_rating_reason                   0
movie_board_rating_display_name       0
movie_release_pattern_display_name    0
dtype: int64

In [34]:
#study the Continuous variables. Except for total earnings column, others dont mean anything
scoring_Sheet.describe(include='all')

,id,name,display_name,production_budget,production_year,movie_sequel,creative_type,source,production_method,genre,language,board_rating_reason,movie_board_rating_display_name,movie_release_pattern_display_name
count,9.100000e+01,91,91,9.100000e+01,91.0,91.000000,91,91,91,91,91,91,91,91
unique,NaN,91,91,NaN,NaN,NaN,8,10,3,9,5,79,5,6
top,NaN,Gone,Gone,NaN,NaN,NaN,Contemporary Fiction,Original Screenplay,Live Action,Drama,English,International - to be excluded,PG-13,Wide
freq,NaN,1,1,NaN,NaN,NaN,45,48,84,26,77,12,31,49
mean,1.607254e+08,NaN,NaN,2.387033e+07,2012.0,0.208791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.756827e+07,NaN,NaN,5.614419e+07,0.0,0.408697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,7.970115e+06,NaN,NaN,0.000000e+00,2012.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.597501e+08,NaN,NaN,0.000000e+00,2012.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.693001e+08,NaN,NaN,0.000000e+00,2012.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.719101e+08,NaN,NaN,1.275000e+07,2012.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#skewness of continuous variables
scoring_Sheet.skew(axis=0)  

id                  -3.771275
production_budget    2.822825
production_year      0.000000
movie_sequel         1.457085
dtype: float64

In [37]:
#Checking the no of rows for each class in categorical variable columns
categorical = scoring_Sheet.iloc[:,[5,6,7,8,9,11,12]]
for c in categorical.columns:
    print("\n")
    print("\n")
    print(c)
    print(categorical[c].value_counts())





movie_sequel
0    72
1    19
Name: movie_sequel, dtype: int64




creative_type
Contemporary Fiction    45
Fantasy                 12
Factual                  9
Historical Fiction       8
Kids Fiction             5
Science Fiction          5
Dramatization            4
Super Hero               3
Name: creative_type, dtype: int64




source
Original Screenplay                    48
Based on Fiction Book/Short Story      17
Based on Real Life Events              11
Based on Factual Book/Article           4
Based on Comic/Graphic Novel            3
Based on Folk Tale/Legend/Fairytale     2
Based on Game                           2
Remake                                  2
Based on TV                             1
Based on Play                           1
Name: source, dtype: int64




production_method
Live Action              84
Digital Animation         5
Animation/Live Action     2
Name: production_method, dtype: int64




genre
Drama                  26
Comedy                 14
Th

In [38]:
# filter stop words based on regular english stop words along with unique words which dont repeat much
#Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
#The board rating reason has several inconsistencies in the text. Manually correcting the spellings etc. This can also be done using TextBlob package
scoring_Sheet.loc[scoring_Sheet['movie_board_rating_display_name'] == 'Not Rated', 'board_rating_reason'] = 'foreign'
board_reason_score = scoring_Sheet['board_rating_reason']
board_reason_score = board_reason_score.str.replace('child','children')
board_reason_score = board_reason_score.str.replace('teen drinking','Underage_drinking')
board_reason_score = board_reason_score.str.replace('teen','teens')
board_reason_score = board_reason_score.str.replace('preteens','teens')
board_reason_score = board_reason_score.str.replace('nudtity','nudity')
board_reason_score = board_reason_score.str.replace('violents','violent')
board_reason_score = board_reason_score.str.replace('sexual attack','molest')
board_reason_score.head()

0    for intense sequences of sci-fi violence and a...
1    for intense sequences of violence and action, ...
2    for intense violent sequences throughout, some...
3    for sequences of violence including disturbing...
4    for extended sequences of intense fantasy acti...
Name: board_rating_reason, dtype: object

In [40]:
board_reason_score = board_reason_score.str.replace('sci-fi action violence','scifi_action_violence')
board_reason_score = board_reason_score.str.replace('sci-fi action and violence','scifi_action_violence')
board_reason_score = board_reason_score.str.replace('sci-fi action','scifi_action')
board_reason_score = board_reason_score.str.replace('sci-fi violence','scifi_violence')
board_reason_score = board_reason_score.str.replace('non-stop','non-stop')
board_reason_score.head()

0    for intense sequences of scifi_violence and ac...
1    for intense sequences of violence and action, ...
2    for intense violent sequences throughout, some...
3    for sequences of violence including disturbing...
4    for extended sequences of intense fantasy acti...
Name: board_rating_reason, dtype: object

In [41]:
board_reason_score = board_reason_score.str.replace('sexaul','sexual')
board_reason_score = board_reason_score.str.replace('PG-13','')
board_reason_score = board_reason_score.str.replace('PG','')
board_reason_score = board_reason_score.str.replace('13','')
board_reason_score = board_reason_score.str.replace('arts','')
board_reason_score = board_reason_score.str.replace('martial','martial_arts')
board_reason_score = board_reason_score.str.replace('-','_')
board_reason_score = board_reason_score.str.replace('kids','children')
board_reason_score

0     for intense sequences of scifi_violence and ac...
1     for intense sequences of violence and action, ...
2     for intense violent sequences throughout, some...
3     for sequences of violence including disturbing...
4     for extended sequences of intense fantasy acti...
                            ...                        
86    for mature thematic material, violence, sexual...
87    for sequences of action violence, thematic ele...
88    for strong sexual content throughout, language...
89    for a scene of violence, brief language and dr...
90                                              foreign
Name: board_rating_reason, Length: 91, dtype: object

In [42]:
English_Stop_Words = text.ENGLISH_STOP_WORDS
Board_Rating_Stop_Words = English_Stop_Words.union(["mild", "crude", "content", "partial", "elements", "comments", "strong", "use", "graphic", 
                                               "mature", "pervasive","and","for","involving", "situations", "caterpillar", "brief",  
                                               "rated", "frequent", "sustained", "pending", "momentary", "re_rated", "image", "edits",
                                               "scene", "situation", "audiences", "inter", "moments", "edited", "version", "originally", 
                                               "images", "including", "references", "suggestive", "thematic", "material","general","intense",
                                               "coarse", "extended", "edition", "realistic", "dequences", "moment", "materials", "descriptions"
                                               ])
vectorizer =  CountVectorizer(stop_words=Board_Rating_Stop_Words)
X = vectorizer.fit_transform(board_reason_score)
print(X)

  (0, 33)	1
  (0, 30)	1
  (0, 1)	1
  (0, 12)	1
  (0, 26)	1
  (1, 33)	1
  (1, 1)	1
  (1, 41)	1
  (1, 31)	1
  (1, 20)	1
  (2, 33)	1
  (2, 20)	1
  (2, 42)	1
  (2, 35)	1
  (2, 36)	1
  (3, 33)	1
  (3, 41)	1
  (3, 31)	1
  (3, 9)	1
  (3, 22)	1
  (4, 33)	1
  (4, 1)	1
  (4, 41)	1
  (4, 14)	1
  (4, 16)	1
  :	:
  (78, 34)	1
  (79, 15)	1
  (80, 15)	1
  (82, 15)	1
  (83, 12)	1
  (83, 42)	1
  (84, 20)	1
  (84, 22)	1
  (85, 15)	1
  (86, 12)	1
  (86, 41)	1
  (86, 20)	1
  (86, 35)	1
  (86, 3)	1
  (87, 33)	1
  (87, 1)	1
  (87, 41)	1
  (87, 20)	1
  (88, 12)	1
  (88, 20)	1
  (88, 34)	1
  (89, 12)	1
  (89, 41)	1
  (89, 20)	1
  (90, 15)	1


In [43]:
print(X.shape)

(91, 44)


In [44]:
# print the vocabulary
print(vectorizer.vocabulary_)

{'sequences': 33, 'scifi_violence': 30, 'action': 1, 'drug': 12, 'reference': 26, 'violence': 41, 'sensuality': 31, 'language': 20, 'violent': 42, 'sexuality': 35, 'smoking': 36, 'disturbing': 9, 'nudity': 22, 'fantasy': 14, 'frightening': 16, 'scary': 29, 'rude': 28, 'humor': 18, 'peril': 24, 'sexual': 34, 'adventure': 2, 'mildly': 21, 'alcohol': 3, 'abuse': 0, 'sequence': 32, 'bloody': 4, 'torture': 39, 'remarks': 27, 'underage_drinking': 40, 'terror': 38, 'brutal': 5, 'grisly': 17, 'domestic': 10, 'children': 6, 'imperilment': 19, 'dramatic': 11, 'partying': 23, 'perilous': 25, 'war': 43, 'depiction': 8, 'extreme': 13, 'dangerous': 7, 'stunts': 37, 'foreign': 15}


In [49]:
scoring_Sheet.columns

Index(['id', 'name', 'display_name', 'production_budget', 'production_year',
       'movie_sequel', 'creative_type', 'source', 'production_method', 'genre',
       'language', 'movie_board_rating_display_name',
       'movie_release_pattern_display_name'],
      dtype='object')

In [50]:
#Dataframe with features extracted from text data
text_features_score = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
#scoring_Sheet.pop('board_rating_reason')
#Concatenating orginal data and the word counts dataframe
merged_Scoring_Data = pd.concat([scoring_Sheet,text_features_score],axis=1)
#Dropping a few columns - Total because we are considering Category as our target, name and displayname are just names, production_year is a 
#categorical variable which keeps changing every year (It can be explored using Panel regression to capture year wise variance.)
merged_Scoring_Data.drop(['id','name','display_name','production_year'], axis=1, inplace=True)
merged_Scoring_Data.head()

,production_budget,movie_sequel,creative_type,source,production_method,genre,language,movie_board_rating_display_name,movie_release_pattern_display_name,abuse,action,adventure,alcohol,bloody,brutal,children,dangerous,depiction,disturbing,domestic,dramatic,drug,extreme,fantasy,foreign,frightening,grisly,humor,imperilment,language,mildly,nudity,partying,peril,perilous,reference,remarks,rude,scary,scifi_violence,sensuality,sequence,sequences,sexual,sexuality,smoking,stunts,terror,torture,underage_drinking,violence,violent,war
0,225000000,0,Super Hero,Based on Comic/Graphic Novel,Live Action,Adventure,English,PG-13,Wide,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,Super Hero,Based on Comic/Graphic Novel,Live Action,Action,English,PG-13,Wide,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
2,200000000,1,Contemporary Fiction,Based on Fiction Book/Short Story,Live Action,Action,English,PG-13,Wide,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0
3,136200000,1,Fantasy,Based on Fiction Book/Short Story,Live Action,Drama,English,PG-13,Wide,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
4,270000000,1,Fantasy,Based on Fiction Book/Short Story,Animation/Live Action,Adventure,English,PG-13,Wide,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


In [51]:
merged_Scoring_Data = merged_Scoring_Data.replace(' ', '_', regex=True)

In [52]:
creative_type = pd.get_dummies(merged_Scoring_Data['creative_type'])
source = pd.get_dummies(merged_Scoring_Data['source'])
production_method = pd.get_dummies(merged_Scoring_Data['production_method'])
genre = pd.get_dummies(merged_Scoring_Data['genre'])
language = pd.get_dummies(merged_Scoring_Data['language'])
movie_board_rating_display_name = pd.get_dummies(merged_Scoring_Data['movie_board_rating_display_name'])
movie_release_pattern_display_name = pd.get_dummies(merged_Scoring_Data['movie_release_pattern_display_name'])

In [53]:
#Joined dummy coding with original data. Then drop original variables
merged_dummies_scoring = pd.concat([merged_Scoring_Data,creative_type,source,production_method,genre,language,movie_board_rating_display_name,movie_release_pattern_display_name],axis=1)
merged_dummies_scoring.drop(['creative_type','source','production_method','genre','language','movie_board_rating_display_name','movie_release_pattern_display_name'], axis=1, inplace=True)

In [57]:
feature_idx = select.get_support()
feature_name = merged_dummies_training.columns[feature_idx]
feature_name

Index(['movie_sequel', 'action', 'bloody', 'disturbing', 'drug', 'foreign',
       'frightening', 'humor', 'nudity', 'rude', 'scifi_action_violence',
       'sensuality', 'sequences', 'sexual', 'sexuality', 'smoking', 'violence',
       'violent', 'Contemporary_Fiction', 'Dramatization', 'Factual',
       'Fantasy', 'Historical_Fiction', 'Kids_Fiction', 'Science_Fiction',
       'Super_Hero', 'Based_on_Comic/Graphic_Novel',
       'Based_on_Factual_Book/Article', 'Based_on_Fiction_Book/Short_Story',
       'Based_on_Real_Life_Events', 'Based_on_TV', 'Original_Screenplay',
       'Remake', 'Animation/Live_Action', 'Digital_Animation', 'Live_Action',
       'Action', 'Adventure', 'Comedy', 'Documentary', 'Drama', 'Horror',
       'Romantic_Comedy', 'Thriller/Suspense', 'language_English', 'G',
       'Not_Rated', 'PG', 'PG-13', 'R', 'Limited', 'Wide'],
      dtype='object')

In [61]:
left = pd.DataFrame(X_train,columns=feature_name)
left.shape

(1667, 52)

In [63]:
headers = left.head(0)
headers


,movie_sequel,action,bloody,disturbing,drug,foreign,frightening,humor,nudity,rude,scifi_action_violence,sensuality,sequences,sexual,sexuality,smoking,violence,violent,Contemporary_Fiction,Dramatization,Factual,Fantasy,Historical_Fiction,Kids_Fiction,Science_Fiction,Super_Hero,Based_on_Comic/Graphic_Novel,Based_on_Factual_Book/Article,Based_on_Fiction_Book/Short_Story,Based_on_Real_Life_Events,Based_on_TV,Original_Screenplay,Remake,Animation/Live_Action,Digital_Animation,Live_Action,Action,Adventure,Comedy,Documentary,Drama,Horror,Romantic_Comedy,Thriller/Suspense,language_English,G,Not_Rated,PG,PG-13,R,Limited,Wide


In [64]:
score_set = pd.concat([headers,merged_dummies_scoring],sort=False)
score_set


,movie_sequel,action,bloody,disturbing,drug,foreign,frightening,humor,nudity,rude,scifi_action_violence,sensuality,sequences,sexual,sexuality,smoking,violence,violent,Contemporary_Fiction,Dramatization,Factual,Fantasy,Historical_Fiction,Kids_Fiction,Science_Fiction,Super_Hero,Based_on_Comic/Graphic_Novel,Based_on_Factual_Book/Article,Based_on_Fiction_Book/Short_Story,Based_on_Real_Life_Events,Based_on_TV,Original_Screenplay,Remake,Animation/Live_Action,Digital_Animation,Live_Action,Action,Adventure,Comedy,Documentary,...,production_budget,abuse,adventure,alcohol,brutal,children,dangerous,depiction,domestic,dramatic,extreme,fantasy,grisly,imperilment,mildly,partying,peril,perilous,reference,remarks,scary,scifi_violence,sequence,stunts,terror,torture,underage_drinking,war,Based_on_Folk_Tale/Legend/Fairytale,Based_on_Game,Based_on_Play,Concert/Performance,language_French,language_Hindi,language_Indonesian/Balinese,language_Spanish,Exclusive,Expands_Wide,IMAX,Special_Engagement
0,0,1,0,0,1,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,...,225000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0,0,0,0,0,0,0,0,NaN,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,0,0,0,0,0,0,0,NaN,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,...,200000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,0,1,0,0,0,0,1,0,NaN,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,136200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0,0,0,0,1,0,0,0,NaN,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,...,270000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,0,0,0,0,1,0,0,0,0,0,NaN,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,0,1,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
88,0,0,0,0,1,0,0,0,0,0,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,0,0,0,0,1,0,0,0,0,0,NaN,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
score_set_final = score_set[feature_name]
score_set_final

,movie_sequel,action,bloody,disturbing,drug,foreign,frightening,humor,nudity,rude,scifi_action_violence,sensuality,sequences,sexual,sexuality,smoking,violence,violent,Contemporary_Fiction,Dramatization,Factual,Fantasy,Historical_Fiction,Kids_Fiction,Science_Fiction,Super_Hero,Based_on_Comic/Graphic_Novel,Based_on_Factual_Book/Article,Based_on_Fiction_Book/Short_Story,Based_on_Real_Life_Events,Based_on_TV,Original_Screenplay,Remake,Animation/Live_Action,Digital_Animation,Live_Action,Action,Adventure,Comedy,Documentary,Drama,Horror,Romantic_Comedy,Thriller/Suspense,language_English,G,Not_Rated,PG,PG-13,R,Limited,Wide
0,0,1,0,0,1,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,1,1,0,0,0,0,0,0,0,0,NaN,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
2,1,0,0,0,0,0,0,0,0,0,NaN,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
3,1,0,0,1,0,0,0,0,1,0,NaN,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1
4,1,1,0,0,0,0,1,0,0,0,NaN,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,0,0,0,0,1,0,0,0,0,0,NaN,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0
87,0,1,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
88,0,0,0,0,1,0,0,0,0,0,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
89,0,0,0,0,1,0,0,0,0,0,NaN,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0


In [66]:
scored_data = score_set_final.replace(np.nan, 0)
scored_data.astype(int)

,movie_sequel,action,bloody,disturbing,drug,foreign,frightening,humor,nudity,rude,scifi_action_violence,sensuality,sequences,sexual,sexuality,smoking,violence,violent,Contemporary_Fiction,Dramatization,Factual,Fantasy,Historical_Fiction,Kids_Fiction,Science_Fiction,Super_Hero,Based_on_Comic/Graphic_Novel,Based_on_Factual_Book/Article,Based_on_Fiction_Book/Short_Story,Based_on_Real_Life_Events,Based_on_TV,Original_Screenplay,Remake,Animation/Live_Action,Digital_Animation,Live_Action,Action,Adventure,Comedy,Documentary,Drama,Horror,Romantic_Comedy,Thriller/Suspense,language_English,G,Not_Rated,PG,PG-13,R,Limited,Wide
0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
3,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1
4,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0
87,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
88,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0
89,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0


In [82]:
y_pred = forest.predict(scored_data)
output = pd.DataFrame(y_pred,columns=['Category_scored'])
output.shape

(91, 1)

In [83]:
scoring_Sheet['id'].shape

(91,)

In [87]:
final = pd.concat([scoring_Sheet['id'],output],axis=1)
final

,id,Category_scored
0,7970115,7
1,174540115,7
2,149080115,7
3,136020115,8
4,53500115,9
...,...,...
86,169020115,1
87,174110115,1
88,169760115,2
89,175970115,2


In [86]:
final.to_excel("scored_dataset.xlsx",
            sheet_name='output')